# Feur_Dashboard


In [1]:
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import *
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [2]:
filename = "Commandes_clients.csv"
df = pd.read_csv(filename)

print(df.head(10))

   IdCommande  IdClient  IdProduit  Date  Quantite
0           1         8          1  1995       225
1           2        14          1  1995       125
2           3        11          9  1995       150
3           4        11          5  1996        25
4           5        11         10  1996       175
5           6         9          8  1996       125
6           7         7          3  1996       250
7           8        12          8  1996       150
8           9        13          3  1996       200
9          10         8          1  1996       200


In [49]:
filename = "Remises.csv"
df_remises = pd.read_csv(filename)
print(df_remises.head(34))

    Remise  IdProduit  IdClient
0        6         10         1
1        4          8        14
2       15          3        11
3       22          5        14
4       12          2         5
5        1          7        11
6       14          9        12
7        5          1         5
8        1          2         9
9       10          1        11
10       9          6         5
11      17          2         6
12      21          2         5
13       2          6         3
14       6          7        13
15       3          8        12
16       2          8        14
17       1          3         2
18       4          5         6
19      14          9         5
20      19          9         7
21       2          8         7
22       5          2        10
23       7          6        11
24       4          2         1
25      19          9         8
26      20          5         7
27      18          9        15
28       9          5        15


In [4]:
filename = "Produits.csv"
df_produit = pd.read_csv(filename)
print(df_produit.head(12))

   Prix  Marge             Nom
0    53     29        Teinture
1    30     30         Ciseaux
2    70     25   Seche_cheveux
3    10     40       Shampoing
4    27     23      Extensions
5    12      9             Gel
6    17      6           Laque
7     5      4           Gants
8    15     20          Brosse
9   750     20  Casque_sechage


In [5]:
filename = "Clients.csv"
df_client = pd.read_csv(filename, encoding='utf-8')
print(df_client)

                    Nom
0            Attrac'tif
1        Communaut'hair
2         Quoi ? Feur !
3              Rela'tif
4           Superla'tif
5            Pendan'tif
6         Helicopt'hair
7             Prec'hair
8          Boulang'hair
9         Poissoni'hair
10     Perpendicul'hair
11            Calc'hair
12  L'atelier du cheveu
13          Centen'hair
14             Addi'tif


     IdCommande  IdClient  IdProduit  Date  Quantite    Cost
8             9        13          3  1996       200   14000
34           35        13          2  1999       150    4500
35           36        13         10  1999       250  187500
39           40        13          9  2000       150    2250
40           41        13         10  2000       150  112500
50           51        13          5  2001        75    2025
76           77        13          1  2005       100    5300
100         101        13          5  2006       200    5400
109         110        13         10  2007       250  187500
119         120        13          6  2009       175    2100
138         139        13          9  2012        75    1125
142         143        13          5  2012       225    6075
143         144        13         10  2012        50   37500
148         149        13          5  2014       150    4050


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Brend\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 14

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Brend\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Brend\AppData\Local\Temp\ipykernel_26760\3211201396.py", line 73, in action
    margeP = df_produit["Marge"][df_commandeUnClient["IdProduit"][i]-1]
  

In [55]:
cost = [df["Quantite"][i]*df_produit["Prix"][df["IdProduit"][i]-1] for i in df.index]

df["Cost"] = cost

df_importance = df.groupby(["IdClient"]).sum().sort_values(by="Cost", ascending=False)


NomClient_importance = [df_client["Nom"][i-1] for i in list(df_importance.index)]
df_importance["Nom"] = NomClient_importance





#def de la fonction qui récupère l'élément selectionné
def action(ev):
    
    #creation des variables qui serviront au calcul de la remise a appliquer
    best = [-1, -1]
    score_max = 0
    
    select = listeCombo.get()
    idselect = list(df_client["Nom"]).index(select)
    
    #instanciation des df pour les utiliser dans l'affichage des remises actuelles
    df_remisesClient = df_remises[df_remises["IdClient"]==idselect]
    
    monTexte=StringVar()
   
    

    #ecriture dans la fenetre tkinter1
    #ecriture des labels d'information pour le client selectionne
    texteLabel = tk.Label(root,text= "Pour le client " + str(df_client["Nom"][idselect]) + ":")
    texteLabel.pack()
    
    if list(df_remisesClient.index):
        for i in list(df_remisesClient.index):
            monTexte.set("Remise actuelle de {}% sur le produit n°{} ({})".format(df_remisesClient["Remise"][i], df_remisesClient["IdProduit"][i], df_produit["Nom"][df_remisesClient["IdProduit"][i]]))
    else:
        monTexte.set("Pas de remise pour ce client")
        
    remiseLabel = tk.Label(root, textvariable= monTexte)
    remiseLabel.pack()
    
    
    #gestion de la fenetre tkinter2
    root2 = tk.Tk()
    fig, axis = plt.subplots(2,2)
    
    
    #pie chart
    total = df["Cost"].sum()
    explosion = [0] * df_importance.shape[0]
    explosion[list(df_importance.index).index(idselect+1)] = 0.15
    camLabel = []
    for i in list(df_importance.index):
        camLabel.append(df_client["Nom"][i-1])
    axis[0,0].pie(df_importance["Cost"], labels = camLabel, explode = explosion)
    
    
    #histogramme
    df_commandeUnClient = df[df["IdClient"]==idselect+1]
    df_histo = df_commandeUnClient.groupby(["Date"]).sum().sort_values(by="Date", ascending=True)
    axis[0,1].bar(height = df_histo["Cost"], x = list(df_histo.index))
    
    #ecriture des informations de conseil de remise a appliquer pour le client selectionne
    print(df_commandeUnClient)
    df_clientPProduit = df_commandeUnClient.groupby(["IdProduit"]).sum()
    for i in list(df_commandeUnClient.count()):
        if not(df_remises["IdProduit"][i] in list(df_remisesClient["IdProduit"])):
            
            margeP = df_produit["Marge"][df_commandeUnClient["IdProduit"][i]-1]
            qteP = df_clientPProduit["Quantite"][df_commandeUnClient["IdProduit"][i]]
            prixP = df_produit["Prix"][df_commandeUnClient["IdProduit"][i]]
            score = margeP * qteP * prixP
            
            if score > score_max:
                score_max = score
                propRemise = rd.randint(1, int(df_produit["Marge"][df_commandeUnClient["IdProduit"][i]])-1)
                best = [df_commandeUnClient["IdProduit"][i], propRemise]

                
    if score_max != 0:
        print("Remise avantageuse détectée !")
        print("Conseil : accordez {}% de remise au client {} sur le produit n°{} ({}), afin de le fidéliser !".format(best[1], nomClient[idselect], best[0], df_produit["Nom"][best[0]]))
    else:
        print("Pas de remise suggérée... :(")
    
    #dessin du canva
    canvas = FigureCanvasTkAgg(fig,master=root2)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    tk.mainloop()

#creation de la combobox
root = tk.Tk()
root.geometry('500x500')

choixLabel = tk.Label(root, text = "Veuillez choisir un client !")

choixLabel.pack()
#ajout de la liste des clients dans la combobox 
listeCombo = ttk.Combobox(root, values=NomClient_importance)

listeCombo.current(0)

listeCombo.pack()
listeCombo.bind("<<ComboboxSelected>>",action)

root.mainloop()

In [ ]:
print(idselect)
df_client = df[df["IdClient"]==idselect]
df_histo = df_client.groupby(["Date"]).sum().sort_values(by="Date", ascending=True)
plt.bar(height = df_histo["Cost"], x = list(df_histo.index))
plt.show()

In [ ]:
df_remisesClient = df_remises[df_remises["IdClient"]==idselect]
print("Pour le client", nomClient[idselect], ":")
if list(df_remisesClient.index):
    for i in list(df_remisesClient.index):
        print("Remise de {}% sur le produit n°{} ({})".format(df_remisesClient["Remise"][i], df_remisesClient["IdProduit"][i], df_produit["Nom"][df_remisesClient["IdProduit"][i]]))
else:
    print("Pas de remise pour ce client")

In [ ]:
best = [-1, -1]
score_max = 0

df_clientPProduit = df_client.groupby(["IdProduit"]).sum()

for i in list(df_client.index):
    if not(df_client["IdProduit"][i] in list(df_remisesClient["IdProduit"])):
        if df_produit["Marge"][df_client["IdProduit"][i]] > 11:
            margeP = df_produit["Marge"][df_client["IdProduit"][i]]
            qteP = df_clientPProduit["Quantite"][df_client["IdProduit"][i]]
            prixP = df_produit["Prix"][df_client["IdProduit"][i]]
            score = margeP * qteP * prixP
            if score > score_max:
                score_max = score
                propRemise = rd.randint(1, int(df_produit["Marge"][df_client["IdProduit"][i]]/10)-1)
                best = [df_client["IdProduit"][i], propRemise * 10]

if score_max != 0:
    print("Remise avantageuse détectée !")
    print("Conseil : accordez {}% de remise au client {} sur le produit n°{} ({}), afin de le fidéliser !".format(best[1], nomClient[idselect], best[0], df_produit["Nom"][best[0]]))
else:
    print("Pas de remise suggérée... :(")